In [108]:
import cv2
import numpy as np
from skimage.filters import frangi, sobel
from matplotlib import pyplot as plt
import pandas as pd

# Część na 3.0

In [2]:
def show_image(img, title="test"):
    cv2.imshow(title, img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

## Wczytanie zdjęcia

In [3]:
#blada gałka oczna na czarnym tle
#image = cv2.imread('zdjecia/2_good.JPG') # 11.42 / 10.99

#bardziej czerwona głka oczna na czrnym tle
#image = cv2.imread('zdjecia/8_good.JPG') # 21.93 / 21.25

#duża pomarańczowa gałka oczna na czarnym tle
image = cv2.imread('zdjecia/Image_03R.jpg') # 51.76 / 56.1

image = cv2.resize(image, (800, 800))

gray_to_find_circle = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

## Wstępne przetworzenie obrazu

In [4]:
def thresh_show(image, th1):
    
    image2 = cv2.threshold(image, th1, 255, cv2.THRESH_BINARY)[1]
    #show_image(image2)
    
    return cv2.findContours(image2, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

In [5]:
#rozmycie

gray_to_find_circle = cv2.GaussianBlur(gray_to_find_circle, (5, 5), 0) # medianblur, guassianblur ?
show_image(image, title="blur")
image = cv2.GaussianBlur(image, (5, 5), 0) # medianblur, guassianblur ?

#wyostrzenie
#obraz potrzebny do odnalezienia gałki ocznej
kernel = np.array([[-1, -1, -1], [-1, 9, -1], [-1, -1, -1]])
gray_to_find_circle = cv2.filter2D(gray_to_find_circle, -1, kernel)
show_image(gray_to_find_circle, title="filter")

#obraz na którym będziemy dalej pracować
#image = cv2.filter2D(image, -1, kernel)
#image = cv2.filter2D(image, -1, kernel)
#show_image(image, title="filter")

image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!
#image = frangi(image, black_ridges=False)
#show_image(image, title="frangi")

In [6]:
# szukamy gałki ocznej
contours, hierarchy = thresh_show(gray_to_find_circle, 0)

contours_approved = []
(cx, cy) , radius = (0, 0), 0
global approxx
maxArea = 0
for cnt in contours:
    approx = cv2.approxPolyDP(cnt, .03 * cv2.arcLength(cnt, True), True)
    if len(approx) >= 7:
        area = cv2.contourArea(cnt)
        (cx, cy), radius = cv2.minEnclosingCircle(cnt)
        circleArea = radius * radius * np.pi
        if maxArea<area:
            contours_approved = []
            maxArea = area
            contours_approved.append(cnt)
            approxx = approx

# wiodący obraz z konturami gałki ocznej i wypełnieniem
cv2.drawContours(image, contours_approved, -1, (255, 0, 0), 3) 
show_image(image, title="wiodacy obraz z konturami galki ocznej")

In [7]:
rgb = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)

fill_color = [255, 255, 255]
mask_value = 255 

stencil  = np.zeros(rgb.shape[:-1]).astype(np.uint8)
cv2.fillPoly(stencil, contours_approved, mask_value)

sel      = stencil != mask_value # select everything that is not mask_value
rgb[sel] = fill_color            # and fill it with fill_color

show_image(rgb, title="wiodacy obraz z konturami galki ocznej i wypelnieniem" )

## Właściwe przetworzenie obrazu

In [8]:
def nothing(x):
    pass

def trackbar_thresholds(image):
    cv2.namedWindow('myImage')

    cv2.createTrackbar('threshold1', 'myImage', 0, 255, nothing)
    cv2.createTrackbar('threshold2', 'myImage', 0, 255, nothing)

    
    image2 = image

    while(1):
        cv2.imshow('myImage', image2)
        k = cv2.waitKey(1) & 0xFF
        if k == 27:
            break
        # get current positions of four trackbars
        th1 = cv2.getTrackbarPos('threshold1', 'myImage')
        th2 = cv2.getTrackbarPos('threshold2', 'myImage')
        # canny
        image2 = cv2.Canny(image, th1, th2)
        
    cv2.destroyAllWindows()

In [9]:
image = cv2.cvtColor(rgb, cv2.COLOR_BGR2GRAY)

In [10]:
def threshold(img, threshpoint):
    return np.array([[1 if x >= threshpoint else 0 for x in y] for y in img])

In [29]:
#trackbar_thresholds(image)

#image = cv2.Canny(image, 100, 230)
# wykrywanie krawędzi
#image = frangi(image, black_ridges=True)
image_mean = np.mean(image)

wspol = 0.7 * image_mean - 10 # 0 - 15
print(wspol)

# thresholding
ret,image2 = cv2.threshold(image,wspol, 255, cv2.THRESH_BINARY)
show_image(image2)



85.7466453125


## Sprawdzenie

In [43]:
odp = cv2.imread('zdjecia/Image_03R_2ndHO.png')
odp = cv2.resize(odp, (800, 800))
odp = cv2.cvtColor(odp, cv2.COLOR_BGR2GRAY)
ret, odp = cv2.threshold(odp,127, 255, cv2.THRESH_BINARY)
odp = cv2.bitwise_not(odp)

#show_image(odp)
max_val = 640000

In [53]:
np.unique(odp)
#show_image(odp)

array([  0, 255], dtype=uint8)

In [102]:
# white == 255 == not vessel
# black == 0 == vessel
global TP, FP, FN, TN

# true-positve (prawdziwie dodatnie)
# naczynie uznane jako naczynie
def count_TP(actually, result):
    i = 0
    for odp_, res_ in zip(actually, result):
        if (odp_ == 0) and (res_ == 0):
            i += 1
    return i

# false-positive (fałszywie dodatnie)
# nie-naczynie uznane jako naczynie
def count_FP(actually, result):
    
    i = 0
    for odp_, res_ in zip(actually, result):
        if (odp_ == 255) and (res_ == 0):
            i += 1
    return i

# false-negative (fałszywie ujemne)
# naczynie uznane jako nie-naczynie
def count_FN(actually, result):
    i = 0
    for odp_, res_ in zip(actually, result):
        if (odp_ == 0) and (res_ == 255):
            i += 1
    return i

# true-negative (prawdziwie ujemne)
# nie-naczynie uznane jako nie-naczynie
def count_TN(actually, result):
    i = 0
    for odp_, res_ in zip(actually, result):
        if (odp_ == 255) and (res_ == 255):
            i += 1
    return i


In [103]:
TP = count_TP( np.nditer(odp), np.nditer(image2))
print("TP = " + str(TP/max_val))

TP = 0.03734375


In [104]:
FP = count_FP( np.nditer(odp), np.nditer(image2))
print("FP = " + str(FP/max_val))

FP = 0.3562390625


In [105]:
FN = count_FN( np.nditer(odp), np.nditer(image2))
print("FN = " + str(FN/max_val))

FN = 0.022509375


In [107]:
TN = count_TN( np.nditer(odp), np.nditer(image2))
print("TN = " + str(TN/max_val))

TN = 0.5839078125


In [114]:
def confusion_matrix():
    names = {
        'My Results' : ['Positive', 'Negative', 'Sum'],
        'Actually - Positive' : [TP, FN, TP + FN],
        'Actually - Negative' : [FP, TN, FP + TN],
        'Sum' : [TP + FP, FN + TN, TP + FP + FN + TN]
    }
    
    return pd.DataFrame(names, columns=['My Results', 'Actually - Positive', 'Actually - Negative', 'Sum' ]).set_index('My Results')

df = confusion_matrix()
df

,Actually - Positive,Actually - Negative,Sum
My Results,,,
Positive,23900,227993,251893
Negative,14406,373701,388107
Sum,38306,601694,640000


In [ ]:
# trafność
def count_accuracy():
    pass

In [ ]:
# czułość
def count_accuracy():
    pass

In [ ]:
# swoistość
def count_accuracy():
    pass

# Część na 5.0

38